In [6]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [ ]:
# train_datagen = ImageDataGenerator(
#     rescale=1.0/255,
#     rotation_range=30,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2  # Split for validation
# )

# train_generator = train_datagen.flow_from_directory(
#     '\content\concrete_crack_images_for_classification.zip',
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode='categorical',
#     subset='training'
# )

# validation_generator = train_datagen.flow_from_directory(
#     '\content\concrete_crack_images_for_classification.zip',
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode='categorical',
#     subset='validation'
# )

# test_datagen = ImageDataGenerator(rescale=1.0/255)
# test_generator = test_datagen.flow_from_directory(
#     '\content\concrete_crack_images_for_classification.zip',
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode='categorical',
#     shuffle=False
# )
import os
import zipfile

# Assuming your zip file is in the current working directory
zip_file_path = '/content/concrete_crack_images_for_classification.zip'  # Correct the path if needed
extract_dir = '/content/extracted_images'  # Choose a directory for extraction

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Update the directory path in your ImageDataGenerator calls
train_generator = train_datagen.flow_from_directory(
    extract_dir,  # Use the extracted directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    extract_dir,  # Use the extracted directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    extract_dir,  # Use the extracted directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# Load the VGG16 model without the top layer (include_top=False)
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model_vgg16.layers:
    layer.trainable = False


In [ ]:
# Add custom layers on top of the base model
x = Flatten()(base_model_vgg16.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
vgg16_model = Model(inputs=base_model_vgg16.input, outputs=output)

# Compile the model
vgg16_model.compile(optimizer=Adam(learning_rate=0.0001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])


In [ ]:
# Train the model
history_vgg16 = vgg16_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


In [ ]:
# Load the ResNet50 model (for testing purposes)
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_resnet = Flatten()(base_model_resnet50.output)
x_resnet = Dense(256, activation='relu')(x_resnet)
x_resnet = Dropout(0.5)(x_resnet)
output_resnet = Dense(train_generator.num_classes, activation='softmax')(x_resnet)
resnet50_model = Model(inputs=base_model_resnet50.input, outputs=output_resnet)
resnet50_model.compile(optimizer=Adam(learning_rate=0.0001),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])


In [ ]:
# Evaluate VGG16 model
vgg16_test_loss, vgg16_test_accuracy = vgg16_model.evaluate(test_generator)
print(f"VGG16 Test Accuracy: {vgg16_test_accuracy:.2f}")

# Evaluate ResNet50 model
resnet50_test_loss, resnet50_test_accuracy = resnet50_model.evaluate(test_generator)
print(f"ResNet50 Test Accuracy: {resnet50_test_accuracy:.2f}")


In [ ]:
# Generate predictions with VGG16
vgg16_predictions = vgg16_model.predict(test_generator)
vgg16_predicted_classes = vgg16_predictions.argmax(axis=1)


In [ ]:
# Generate predictions with ResNet50
resnet50_predictions = resnet50_model.predict(test_generator)
resnet50_predicted_classes = resnet50_predictions.argmax(axis=1)


In [ ]:
import numpy as np

# Compare the predictions of VGG16 and ResNet50
comparison = np.equal(vgg16_predicted_classes, resnet50_predicted_classes)
print(f"Number of matching predictions: {np.sum(comparison)} / {len(comparison)}")
